# Deliverable 2
This is the notebook that has the second deliverable.

# 0. Import modules
Feel free to use the virtual environment (amonavis) included in the GitHub folder.

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt 
import networkx as nx

# For semantic similarity
from urllib.parse import unquote
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Python functions in .py file to read data
import data_readers

# 1. Data reading
The following code reads the data and saves them in the appropriate variables.

<br><br><br>
**finished_paths**

The datafile includes the hashedIpAddress, timestamp, durationInSec, path, and rating.

We also add columns with the first article (soruce), last article (target), path length (#articles), and a readable date in Timestamp format.

<br><br>
**articles**

Dataframe with the name of all articles in the dataset.


<br><br><br>


<div style="border: 2px solid white; padding: 10px;">
    <font color="red">please briefly describe the other files. As I only worked with finished_paths and articles, I guess it's better if someone who used them does that. Also add a sample of how each data looks like. </font>
</div>

In [2]:
# The links and edges
wikispeedia = data_readers.read_wikispeedia_graph()

# The finished paths
finished_paths = data_readers.read_finished_paths()

# The unfinished paths
unfinished_paths = data_readers.read_unfinished_paths()

# List of all articles
articles = data_readers.read_articles()

# We found out later that the data contained in the shortest path matrix given to us seems to be wrong
# Here we also add a quick dictionary that properly shows that this is wrong, and give an example
shortest_path_df = data_readers.read_shortest_path_df()
shortest_path_dict = dict(nx.all_pairs_shortest_path(wikispeedia))

# Searching for the string of a given article. It has to be formatted like the article name
# Which shouldn't be a problem, as we'll probably usually retrieve them internally
obi_wan_text = data_readers.plaintext_article_finder('Obi-Wan_Kenobi')

In [3]:
print("Dataset has", len(wikispeedia.nodes), "nodes")
print("Dataset has", len(wikispeedia.edges), "edges")

Dataset has 4592 nodes
Dataset has 119882 edges


These are less nodes than the reported number, it should be 4,604 nodes.

The 119,882 edges is correct though.

The difference is probably due to some nodes not being connected to the rest of the graph, as here we read in only the articles that are connected. The few nodes that we are losing do not matter for what we need

In [4]:
finished_paths.sample(5)

,hashedIpAddress,timestamp,durationInSec,path,rating,first_article,last_article,path_length,date
23203,461a9f80797f6406,1363697997,429,Formic_acid;Alchemy;Frankenstein;Popular_cultu...,NaN,Formic_acid,John_F._Kennedy_assassination,9,2013-03-19 12:59:57
2968,28bd91e879d8ea9d,1344855924,57,Cat;Dutch_language;Netherlands;Nazi_Germany;Ad...,NaN,Cat,Adolf_Hitler,5,2012-08-13 11:05:24
6351,18905b8f2635f996,1372167354,32,Mormon;Christianity;Southern_Africa;Democratic...,1.0,Mormon,Republic_of_the_Congo,5,2013-06-25 13:35:54
49607,5cc47df267db4d3f,1248727556,51,Troodon;North_America;United_States;President_...,1.0,Troodon,James_K._Polk,5,2009-07-27 20:45:56
31818,12c3d3316114fb01,1302276264,20,Oppression;Psychology;Biology;Organism,1.0,Oppression,Organism,4,2011-04-08 15:24:24


In [5]:
unfinished_paths.sample(5)

,hashedIpAddress,timestamp,durationInSec,path,target,type
18904,206c212604a78fb1,1368019245,73,Battle_of_Normandy;Nazi_Germany;Adolf_Hitler;J...,Reza_Shah,restart
5871,106a395641982357,1317225145,1911,Magic__The_Gathering;Board_game;Iran;Islam;Mid...,Al_Jazeera,timeout
13498,1e1846581384e33e,1350893596,107,Statue_of_Zeus_at_Olympia;Greece;Climate;Weath...,Pumpkin,restart
20506,4d8e50314258ad1d,1374008831,6,Visual_arts,Parsley,restart
10581,576357c67f621837,1344900692,1900,Brain;Computer_science;Internet;Advertising,Telephone,timeout


In [6]:
articles.sample(5)

,articles
3514,Robert_Oppenheimer
2224,Jane_Eyre
4103,Theatre_Royal%2C_Drury_Lane
811,Carlsbad_Caverns_National_Park
3428,Raising_the_Flag_on_Iwo_Jima


In [7]:
shortest_path_df.head()

,"(%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,)","(%C3%85land,)","(%C3%89douard_Manet,)","(%C3%89ire,)","(%C3%93engus_I_of_the_Picts,)","(%E2%82%AC2_commemorative_coins,)","(10th_century,)","(11th_century,)","(12th_century,)","(13th_century,)",...,"(Ziad_Jarrah,)","(Zimbabwe,)","(Zinc,)","(Zinc_chloride,)","(Zion_National_Park,)","(Zionism,)","(Zirconium,)","(Zoroaster,)","(Zuid-Gelders,)","(Zulu,)"
"(%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,)",0,-1,-1,-1,-1,-1,3,3,3,3,...,4,3,3,4,4,3,4,4,4,2
"(%C3%85land,)",-1,0,-1,-1,-1,-1,2,2,2,2,...,4,2,3,4,4,3,4,3,3,3
"(%C3%89douard_Manet,)",-1,-1,0,-1,-1,-1,3,3,2,2,...,4,3,2,3,4,3,4,3,3,3
"(%C3%89ire,)",-1,-1,-1,0,-1,-1,3,3,3,3,...,4,2,2,3,4,3,4,4,3,3
"(%C3%93engus_I_of_the_Picts,)",-1,-1,-1,-1,0,-1,2,2,3,2,...,4,2,3,4,4,3,4,3,3,3


## Issue with the shortest path
The shortest paths that were calculated are wrong. This is just a simple example to show that at least one of the points is wrong. We assume this is common enough elsewhere, as the effort needed to properly prove this is too large.

We double-checked the method for reading in the data, and there are no big changes to be done there. We also double-checked that the graph is a directed graph, which matters for this search. Additionally, we manually checked the edges, so we know that the following example is correct.

In [23]:
print("Shortest path from Actor to Japan in the given matrix is:", shortest_path_df[('Actor',)][('Japan',)])

print("Shortest path from Actor to Japan according to networkX is:", len(shortest_path_dict['Actor']['Japan'])-1)

print("The actual path is:", shortest_path_dict['Actor']['Japan'])

Shortest path from Actor to Japan in the given matrix is: 3
Shortest path from Actor to Japan according to networkX is: 1
The actual path is: ['Actor', 'Japan']


In [9]:
# No need to print the entire text, just show that reading it in works
obi_wan_text[:100]

'   #copyright\n\nObi-Wan Kenobi\n\n2007 Schools Wikipedia Selection. Related subjects: Films\n\n   Star Wa'

# 2. Study of Unique Paths
Here we study the unique source and target pairs. We will use the dataframes to compare the performance between humans and machines, as well as to know what paths to make machines complete.

**article_combinations**

This dataframe contains information on all the combination of source and target articles in the finished games (paths). It includes how many times it has been played, and the mean and std of the path length, duration of the game, and rating.

**unique_targets** and **unique_sources**

These dataframes include all the sources and targets that appears in the finished games

<br><br>
Note that we don't change to ASCII the name of the articles yet. We will do it at a later step if we need to.
<br><br>


<div style="border: 2px solid white; padding: 10px;">
    <font color="red">feel free to move this wherever works better. Maybe not the best thing to have at the beginning of the notebook.</font>
</div>

In [10]:
# How many each pair of articles has been visited
article_combinations_count = finished_paths.groupby(['first_article', 'last_article']).size().reset_index(name='count')

# The mean and std of the path length for each pair of articles
article_combinations_stats = finished_paths.groupby(['first_article', 'last_article'])['path_length'].agg(['mean', 'std']).reset_index()
article_combinations_stats['std'] = article_combinations_stats['std'].fillna(0)
article_combinations_stats.rename(columns={'mean': 'mean_length', 'std': 'std_length'}, inplace=True)

# The mean and std of the rating for each pair of articles. 
    # Note that mean and std may be nan if there are nan ratings. We purposely leave them as nan, as we don't want to fill them with 0s or 1s.
    # Depending on the application, we could change this in the future if neeeded.
rating_combinations_stats_rating = finished_paths.groupby(['first_article', 'last_article'])['rating'].agg(['mean', 'std']).reset_index()
#rating_combinations_stats_rating['std'] = rating_combinations_stats_rating['std'].fillna(0)
mask = rating_combinations_stats_rating['mean'].notnull()
rating_combinations_stats_rating.loc[mask, 'std'] = rating_combinations_stats_rating.loc[mask, 'std'].fillna(0)
rating_combinations_stats_rating.rename(columns={'mean': 'mean_rating', 'std': 'std_rating'}, inplace=True)

# The mean and std of the time for each pair of articles.
rating_combinations_stats_time = finished_paths.groupby(['first_article', 'last_article'])['durationInSec'].agg(['mean', 'std']).reset_index()
rating_combinations_stats_time['std'] = rating_combinations_stats_time['std'].fillna(0)
rating_combinations_stats_time.rename(columns={'mean': 'mean_durationInSec', 'std': 'std_durationInSec'}, inplace=True)

# Merging all the dataframes
article_combinations = pd.merge(article_combinations_count, article_combinations_stats, on=['first_article', 'last_article'])
article_combinations = pd.merge(article_combinations, rating_combinations_stats_rating, on=['first_article', 'last_article'])
article_combinations = pd.merge(article_combinations, rating_combinations_stats_time, on=['first_article', 'last_article'])

# The number of unique sources and targets
unique_sources = finished_paths['first_article'].value_counts().reset_index()
unique_targets = finished_paths['last_article'].value_counts().reset_index()

In [11]:
article_combinations.sample(5)

,first_article,last_article,count,mean_length,std_length,mean_rating,std_rating,mean_durationInSec,std_durationInSec
17040,Meteorological_history_of_Hurricane_Katrina,Quantity,2,9.5,3.535534,3.0,0.0,244.0,22.627417
27458,Vole,Strait_of_Malacca,1,8.0,0.000000,NaN,NaN,102.0,0.000000
26961,University_of_Chicago,Reggaeton,1,3.0,0.000000,1.0,0.0,64.0,0.000000
4153,Bob_Marley,Earth,1,4.0,0.000000,NaN,NaN,33.0,0.000000
9887,Florida,The_Championships%2C_Wimbledon,1,8.0,0.000000,3.0,0.0,111.0,0.000000


In [12]:
unique_sources.sample(5)

,first_article,count
1824,Jerusalem,10
2995,Pellagra,6
997,Conflict,15
2321,Morecambe_Bay,8
2188,Canterbury,9


In [13]:
unique_targets.sample(5)

,last_article,count
1536,Romanticism,10
2330,RER,6
2201,Oligarchy,7
2553,Great_Spotted_Woodpecker,6
1526,Volcanic_pipe,10


# 3. Semantic Similarity

An important part of the project is to study how humans and machines move from article to article. Semantic similarity compares two strings based on a trained model and assigns a value according to how are they correlated (the higher, the more related). For example, 'king' and 'queen' will have a higher semantic similarity than say, 'king' and 'chemistry' (will prove this here).

### Remove the underscore and decode the url

First we define a function that corrects the strings to have a readable format. For example, '%C3%89douard_Manet' is transformed to 'Édouard Manet'.

We will create a function to decode a word, and we will be able to use it in series and dataframes using apply().

In [14]:
def decode_word(word):
    word = word.replace('_', ' ')
    return unquote(word)

In [15]:
articles['articles'].head(5)

0    %C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1                          %C3%85land
2                  %C3%89douard_Manet
3                           %C3%89ire
4          %C3%93engus_I_of_the_Picts
Name: articles, dtype: object

In [16]:
articles['articles'].apply(decode_word).head(5)

0        Áedán mac Gabráin
1                    Åland
2            Édouard Manet
3                     Éire
4    Óengus I of the Picts
Name: articles, dtype: object

### Semantic Distance Model
We create a function that returns the semantic similarity between two words you provide.

In [17]:
# We define the model outside the function (make sure to run this before using the function)
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [18]:
# Function to get embeddings (just because we will use it in semantic similarity function)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Semantic similarity function
def semantic_similarity(word1, word2):
    embedding1 = get_embedding(word1)
    embedding2 = get_embedding(word2)
    return cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]

In [19]:
semantic_similarity('king', 'queen')

0.9388243

In [20]:
semantic_similarity('king', 'chemistry')

0.6550222

### Semantic Distance Matrix
Provided a series, it creates a df where indices and column names are the strings of the series, and fills the matrix with the semantic similarity between all words in the provided series.

In [21]:
def semantic_similarity_matrix(titles):
    df = pd.DataFrame(index=titles, columns=titles)
    for i in range(len(titles)):
        for j in range(i+1, len(titles)):
            embedding1 = get_embedding(titles[i])
            embedding2 = get_embedding(titles[j])
            similarity = cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]
            df.iloc[i, j] = similarity
            df.iloc[j, i] = similarity  # Copy value to lower triangle
            np.fill_diagonal(df.values, 1)
    return df

In [22]:
semantic_similarity_matrix(pd.Series(['king', 'queen', 'chemistry', 'biology']))

,king,queen,chemistry,biology
king,1,0.938824,0.655022,0.734002
queen,0.938824,1,0.633365,0.745616
chemistry,0.655022,0.633365,1,0.846376
biology,0.734002,0.745616,0.846376,1


# 4. Basic AI

Most of the model requires for there to be an AI model we can compare it against.

We were indicated by the TA to not focus on this too much, as this is a data analysis course, not an ML course. Because of this, we took the implementation of A\* that was included in networkX and created two modified versions that now do the following:
- First version returns all of the explored nodes, not just the shortest path found
- Second version is forced to do a depth first search without being able to return

For our purposes, the explored nodes is the most interesting metric, as it describes what were the links "clicked".

We found a paper that implements a more complicated version, and we might be able to do something with graph neural networks, but for now this is good enough.

We additionally do a bit of work to show how the system works timewise, as well as how the comparison will work in the future.

The time comparison is not super useful though, as that will depend on hardware too much to be worth using easily.

In [30]:
import machine_searchers
import time

def modded_get_embedding(text: str):
    temp_str = text.replace('_', ' ')
    temp_str = unquote(temp_str)
    inputs = tokenizer(temp_str, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def distance_two_words(w1: str, w2: str):
    """Receives a string that was in the wikispeedia dataset, and transforms it as needed to work
    with the berd embeddings."""

    embedding1 = modded_get_embedding(w1)
    embedding2 = modded_get_embedding(w2)
    similarity = cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]
    # Adding absolute, just in case it is needed
    # Similarity is actually 1 - abs(similarity) + 1,
    # As we want closer words to have a smaller distance
    # The last plus one is to indicate that there would be an extra cost to exploring, as if not the system often
    # thinks that there are nodes that have a distance of 0.5 or something like that
    similarity = 1 - abs(similarity) + 1
    # print("First word:", w1, ". Second word:", w2, ". GoodDistance:", similarity)
    return similarity

start_time = time.time()
lib_path_1, lib_explore_1 = machine_searchers.modded_astar_path(wikispeedia, 'Actor', 'Japan', heuristic=distance_two_words)
end_time = time.time()

# It's len - 1 because the target node is also included, and that node wasn't explored
print("Using the modded a star that returns explored nodes:")
print(" Found solution for Actor to Japan exploring the following number of nodes:", len(lib_explore_1)-1)
print(" Found it in:", end_time-start_time)

start_time = time.time()
lib_path_2, lib_explore_2 = machine_searchers.only_depth_first_astar_path(wikispeedia, 'Actor', 'Japan', heuristic=distance_two_words)
end_time = time.time()

# It's len - 1 because the target node is also included, and that node wasn't explored
print("Using depth first only A star that returns explored nodes:")
print(" Found solution for Actor to Japan exploring the following number of nodes:", len(lib_explore_1)-1)
print(" Found it in:", end_time-start_time)


Using the modded a star that returns explored nodes:
 Found solution for Actor to Japan exploring the following number of nodes: 1
 Found it in: 1.0417146682739258
Using depth first only A star that returns explored nodes:
 Found solution for Actor to Japan exploring the following number of nodes: 1
 Found it in: 0.6542062759399414


Now we'll take the most commonly explored node pair path, run it through the two algorithms and see what is the result!

In [35]:
# So it's finding the length between asteroid and viking
article_combinations.sort_values('count', ascending=False).head()

,first_article,last_article,count,mean_length,std_length,mean_rating,std_rating,mean_durationInSec,std_durationInSec
2318,Asteroid,Viking,1043,7.516779,3.019205,2.554455,0.980963,202.216683,281.960852
4427,Brain,Telephone,1040,7.100000,3.580183,2.482051,1.043248,174.138462,189.381553
25700,Theatre,Zebra,905,7.836464,3.849462,2.635359,1.044531,251.068508,1169.866464
21255,Pyramid,Bean,642,8.246106,4.259726,2.677419,1.010459,199.085670,191.480244
3206,Batman,Wood,148,7.263514,3.413382,2.851852,1.199478,113.466216,141.024406


In [37]:
start_time = time.time()
lib_path_1, lib_explore_1 = machine_searchers.modded_astar_path(wikispeedia, 'Asteroid', 'Viking', heuristic=distance_two_words)
end_time = time.time()

# It's len - 1 because the target node is also included, and that node wasn't explored
print("Using the modded a star that returns explored nodes:")
print(" Found solution for Asteroid to Viking exploring the following number of nodes:", len(lib_explore_1)-1)
print("Path length was:", len(lib_path_1)-1)
print(" Found it in:", end_time-start_time)

start_time = time.time()
lib_path_2, lib_explore_2 = machine_searchers.only_depth_first_astar_path(wikispeedia, 'Asteroid', 'Viking', heuristic=distance_two_words)
end_time = time.time()
print('')

# It's len - 1 because the target node is also included, and that node wasn't explored
print("Using depth first only A star that returns explored nodes:")
print(" Found solution for Asteroid to Viking exploring the following number of nodes:", len(lib_explore_2)-1)
print("Path length was:", len(lib_path_2)-1)
print(" Found it in:", end_time-start_time)

Using the modded a star that returns explored nodes:
 Found solution for Asteroid to Viking exploring the following number of nodes: 52
Path length was: 3
 Found it in: 80.1152811050415
Using depth first only A star that returns explored nodes:
 Found solution for Asteroid to Viking exploring the following number of nodes: 40
Path length was: 40
 Found it in: 510.53669261932373


In [38]:
lib_path_1

['Asteroid', '1_Ceres', 'Paris', 'Viking']

So, based on the previous example of the path, this works out well enough. The systems spends a lot of time exploring and going back, which might be a common issue. There is a huge disconnect between explored and actual path length, but that is common for A\*, so it's an expected caveat.

It is interesting to note that the optimal solution passed through Paris, which seems to fit the definition of being one of the hubs that are described in the paper. Maybe the hub strategy is actually useful in most cases!

The depth first method took a lot longer to run than planned. Based on this, it might be worth considering other alternatives.

But still, at least we've proven the model works, and can give results that we can compare against humans. Again, this is using a much simpler method, but this could be enhanced in the future.